# Spam Classifier
## Assignment Preamble
Please ensure you carefully read all of the details and instructions on the assignment page, this section, and the rest of the notebook. If anything is unclear at any time please post on the forum or ask a tutor well in advance of the assignment deadline.

In addition to all of the instructions in the body of the assignment below, you must also follow the following technical instructions for all assignments in this unit. *Failure to do so may result in a grade of zero.*
* [At the bottom of the page](#Submission-Test) is some code which checks you meet the submission requirements. You **must** ensure that this runs correctly before submission.
* Do not modify or delete any of the cells that are marked as test cells, even if they appear to be empty.
* Do not duplicate any cells in the notebook – this can break the marking script. Instead, insert a new cell (e.g. from the menu) and copy across any contents as necessary.

Remember to save and backup your work regularly, and double-check you are submitting the correct version.

This notebook is the primary reference for your submission. You may write code in separate `.py` files but it must be clearly imported into the notebook so that it runs without needing to reference those files, and you must explain clearly what functionality is contained in those files (through comments, markdown cells, etc).

As always, **the work you submit for this assignment must be entirely your own.** Do not copy or work with other students. Do not copy answers that you find online. These assignments are designed to help improve your understanding first and foremost – the process of doing the assignment is part of *learning*. They are also used to assess your ability, and so you must uphold academic integrity. Submitting plagiarised work risks your entire place on your degree.

**The pass mark for this assignment is 40%.** We expect that students, on average, will be able to produce a submission which gets a mark between 50-70% within the normal workload allocation for the unit, but this will vary depending on individual backgrounds. Please ask for help if you are struggling.

## Getting Started
Spam refers to unwanted email, often in the form of advertisements. In the literature, an email that is **not** spam is called *ham*. Most email providers offer automatic spam filtering, where spam emails will be moved to a separate inbox based on their contents. Of course this requires being able to scan an email and determine whether it is spam or ham, a classification problem. This is the subject of this assignment.

This assignment has two parts. Each part is worth 50% of the overall grade for this assignment.

For part one you will write a supervised learning based classifier to determine whether a given email is spam or ham. You must write and submit the code in this notebook. The training data is provided for you. You may use any classification method. Marks will be awarded primarily based on the accuracy of your classifier on unseen test data, but there are also marks for estimating how accurate you think your classifier will be.

In part two you will produce a short video explaining your implementation, any decisions or extensions you made, and what parameter values you used. This part is explained in more detail on the assignment page. The video file must be submitted with your assignment.

### Choice of Algorithm
While the classification method is a completely free choice, the assignment folder includes [a separate notebook file](data/naivebayes.ipynb) which can help you implement a Naïve Bayes solution. If you do use this notebook, you are still responsible for porting your code into *this* notebook for submission. A good implementation should give a high  enough accuracy to get a good grade on this section (50-70%).

You could also consider a k-nearest neighbour algorithm, but this may be less accurate. Logistic regression is another option that you may wish to consider.

If you are looking to go beyond the scope of the unit, you might be interested in building something more advanced, like an artificial neural network. This is possible just using `numpy`, but will require significant self-directed learning. *Extensions like this are left unguided and are not factored into the unit workload estimates.*

**Note:** you may use helper functions in libraries like `numpy` or `scipy`, but you **must not** import code which builds entire models for you. This includes but is not limited to use of libraries like `scikit-learn`, `tensorflow`, or `pytorch` – there will be plenty of opportunities for these libraries in later units. The point of this assignment is to understand code the actual algorithm yourself. ***If you are in any doubt about any particular library or function please ask a tutor.*** Submissions which ignore this will receive penalties or even zero marks.

If you choose to implement more than one algorithm, please feel free to include your code and talk about it in part two (your video presentation), but only the code in this notebook will be used in the automated testing.

## Training Data
The training data is described below and has 1000 rows. There is also a 500 row set of test data. These are functionally identical to the training data, they are just in a separate csv file to encourage you to split out your training and test data. You should consider how to best make use of all available data without overfitting, and to help produce an unbiased estimate for your classifier's accuracy.

The cell below loads the training data into a variable called `training_spam`.

In [1]:
import numpy as np

training_spam = np.loadtxt(open("data/training_spam.csv"), delimiter=",").astype(np.int)
print("Shape of the spam training data set:", training_spam.shape)
print(training_spam)

Shape of the spam training data set: (1000, 55)
[[1 0 0 ... 0 0 0]
 [0 0 1 ... 1 0 0]
 [0 0 0 ... 1 0 0]
 ...
 [0 0 0 ... 0 0 1]
 [1 1 1 ... 1 1 0]
 [1 0 0 ... 1 1 1]]


Your training set consists of 1000 rows and 55 columns. Each row corresponds to one email message. The first column is the _response_ variable and describes whether a message is spam `1` or ham `0`. The remaining 54 columns are _features_ that you will use to build a classifier. These features correspond to 54 different keywords (such as "money", "free", and "receive") and special characters (such as ":", "!", and "$"). A feature has the value `1` if the keyword appears in the message and `0` otherwise.

As mentioned there is also a 500 row set of *test data*. It contains the same 55 columns.

In [2]:
testing_spam = np.loadtxt(open("data/testing_spam.csv"), delimiter=",").astype(np.int)
print("Shape of the spam testing data set:", testing_spam.shape)
print(testing_spam)

Shape of the spam testing data set: (500, 55)
[[1 0 0 ... 1 1 1]
 [1 1 0 ... 1 1 1]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 1 0 0]
 [0 0 0 ... 1 0 0]]


## Part One
Write all of the code for your classifier below this cell. There is some very rough skeleton code in the cell directly below. You may insert more cells below this if you wish, but you must not duplicate any cells as this can break the grading script.

### Submission Requirements
Your code must provide a variable with the name `classifier`. This object must have a method called `predict` which takes input data and returns class predictions. The input will be a single $n \times 54$ numpy array, your classifier should return a numpy array of length $n$ with classifications. There is a demo in the cell below, and a test you can run before submitting to check your code is working correctly.

Your code must run on our test machine in under 30 seconds. If you wish to train a more complicated model (e.g. neural network) which will take longer, you are welcome to save the model's weights as a file and then load these in the cell below so we can test it. You must include the code which computes the original weights, but this must not run when we run the notebook – comment out the code which actually executes the routine and make sure it is clear what we need to change to get it to run. Remember that we will be testing your final classifier on additional hidden data.

In [54]:
# This skeleton code simply classifies every input as ham
#
# Here you can see there is a parameter k that is unused, the
# point is to show you how you could set up your own. You might
# also pass in extra data via a train method (also does nothing
# here). Modify this code as much as you like so long as the 
# accuracy test in the cell below runs.

import pickle
import numpy as np
from venv.dense import DenseLayer
from venv.activations import Sigmoid, Softmax
from venv.losses import binary_cross_entropy, binary_cross_entropy_deriv

# Original train dataset 
X_train = np.reshape([training_spam[:,1:]], (1000, 54, 1))
Y_train = np.reshape([training_spam[:,:1]], (1000, 1, 1))

# Original test dataset
X_test = np.reshape([testing_spam[:,1:]], (500, 54, 1))
Y_test = np.reshape([testing_spam[:,:1]], (500, 1, 1))


# Combine train and test to to final fit of model weights
X_train = np.vstack((X_train, X_test))
Y_train = np.vstack((Y_train, Y_test))
              

np.random.seed(1)

network = [
    DenseLayer(54, 15),
    Sigmoid(),
    DenseLayer(15, 10),
    Sigmoid(),
    DenseLayer(10, 5),
    Sigmoid(),
    DenseLayer(5, 1),
    Sigmoid()
]

epochs = 500
learning_rate = 0.0005


class SpamClassifier:

    def train(self, epochs, X_train, Y_train, network, binary_cross_entropy,
              binary_cross_entropy_deriv, learning_rate):
        
        def forward_output(network, input):
            output = input
            for layer in network:
                output = layer.forward_prop(output)
            return output
        
        for e in range(epochs):
            error = 0
            for x, y in zip(X_train, Y_train):
                output = x

                for layer in network:
                    output = layer.forward_prop(output)

                # Forward prop 
                output = forward_output(network, x)

                # Error
                error += binary_cross_entropy(y, output)

                # Back prop
                grad = binary_cross_entropy_deriv(y, output)
                for layer in reversed(network):
                    grad = layer.back_prop(grad, learning_rate)

            error /= len(X_train)
            print(f"{e + 1}/{epochs}, accuracy={(error) } ")

        
        # Save network weights 
        saved_network = []
        for key, i in enumerate(network):
            if key % 2 == 0:
                saved_network.append(i)
        
        with open("network_weights", "wb") as fp:
            
            pickle.dump(saved_network, fp)
        
        return saved_network
        
        
    def predict(self, data):
        
        # Reshape data
        data = np.reshape([data], (data.shape[0], data.shape[1], 1))
        
        # Load weights 
        
        with open("network_weights", "rb") as fp:
            saved_network = pickle.load(fp)

        # Re-build neural network
        for key, i in enumerate(saved_network):
            if key % 2 != 0:
                saved_network.insert(key, Sigmoid())
        saved_network.append(Sigmoid())

        count = 0
        tot_output = []
        
        # Loop through data
        for x in data: 
            output = x
            # Propagate output through the network 
            for layer in saved_network:
                output = layer.forward_prop(output)
            
            # Classify output
            if output.item() <= 0.5:
                output = 0
            else:
                output = 1
            
            tot_output.append(output)
        tot_output = np.asarray(tot_output)
        
        return tot_output
    

def create_classifier():
    classifier = SpamClassifier()
    
    """
    Uncomment these two lines to train the model. Currently using pre-trained model.
    
    classifier.train(epochs, X_train, Y_train, network, binary_cross_entropy, 
                     binary_cross_entropy_deriv, learning_rate)"""

    classifier.predict(X_test)
    return classifier

classifier = create_classifier()
classifier.predict(X_test)

1/500, accuracy=1.2932699123740607 
2/500, accuracy=0.851361480802541 
3/500, accuracy=0.5071243729718549 
4/500, accuracy=0.2912149426028532 
5/500, accuracy=0.17411957875544165 
6/500, accuracy=0.11481363917678777 
7/500, accuracy=0.08559718053506205 
8/500, accuracy=0.07135003531280067 
9/500, accuracy=0.06441337781507951 
10/500, accuracy=0.06101755141123129 
11/500, accuracy=0.0593290125962752 
12/500, accuracy=0.05846046201825625 
13/500, accuracy=0.05798342886182761 
14/500, accuracy=0.05769114668767743 
15/500, accuracy=0.05748379579239337 
16/500, accuracy=0.057313089481256164 
17/500, accuracy=0.05715553909608543 
18/500, accuracy=0.05699955719097811 
19/500, accuracy=0.05683923322473577 
20/500, accuracy=0.056671325293370454 
21/500, accuracy=0.05649380144858983 
22/500, accuracy=0.056305127662022364 
23/500, accuracy=0.056103916143321574 
24/500, accuracy=0.05588874892427394 
25/500, accuracy=0.05565808896365401 
26/500, accuracy=0.055410238267133656 
27/500, accuracy=0.055

214/500, accuracy=0.013068026372250765 
215/500, accuracy=0.013038341961323452 
216/500, accuracy=0.013009238277512014 
217/500, accuracy=0.01298069007759692 
218/500, accuracy=0.01295267335687461 
219/500, accuracy=0.012925165329460765 
220/500, accuracy=0.012898144407244651 
221/500, accuracy=0.012871590177871478 
222/500, accuracy=0.012845483382092442 
223/500, accuracy=0.01281980589077485 
224/500, accuracy=0.012794540681784303 
225/500, accuracy=0.012769671816963828 
226/500, accuracy=0.012745184419366675 
227/500, accuracy=0.012721064650851306 
228/500, accuracy=0.012697299690132634 
229/500, accuracy=0.012673877711340896 
230/500, accuracy=0.01265078786311131 
231/500, accuracy=0.012628020248192944 
232/500, accuracy=0.012605565903555025 
233/500, accuracy=0.01258341678093691 
234/500, accuracy=0.012561565727771501 
235/500, accuracy=0.012540006468387894 
236/500, accuracy=0.012518733585423228 
237/500, accuracy=0.012497742501293475 
238/500, accuracy=0.01247702945964084 
239/50

421/500, accuracy=0.012778148931612505 
422/500, accuracy=0.01278598052083271 
423/500, accuracy=0.012793804847851644 
424/500, accuracy=0.01280162127315776 
425/500, accuracy=0.0128094291477814 
426/500, accuracy=0.012817227814034358 
427/500, accuracy=0.012825016606274402 
428/500, accuracy=0.012832794851643615 
429/500, accuracy=0.012840561870807848 
430/500, accuracy=0.012848316978693748 
431/500, accuracy=0.012856059485199864 
432/500, accuracy=0.012863788695925634 
433/500, accuracy=0.012871503912864757 
434/500, accuracy=0.01287920443510423 
435/500, accuracy=0.012886889559504679 
436/500, accuracy=0.012894558581382585 
437/500, accuracy=0.012902210795159123 
438/500, accuracy=0.012909845495012635 
439/500, accuracy=0.012917461975517084 
440/500, accuracy=0.012925059532261744 
441/500, accuracy=0.012932637462464728 
442/500, accuracy=0.012940195065565244 
443/500, accuracy=0.01294773164380939 
444/500, accuracy=0.012955246502813602 
445/500, accuracy=0.012962738952118724 
446/50

array([1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0,
       0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1,
       0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1,
       1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0,
       0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1,
       0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0,
       1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1,

### Accuracy Estimate
In the cell below there is a function called `my_accuracy_estimate()` which returns `0.5`. Before you submit the assignment, write your best guess for the accuracy of your classifier into this function, as a percentage between `0` and `1`. So if you think you will get 80% of inputs correct, return the value `0.8`. This will form a small part of the marking criteria for the assignment, to encourage you to test your own code.

In [61]:
def my_accuracy_estimate():
    accuracy = 0.915
    return accuracy

Write all of the code for your classifier above this cell.

### Testing Details
Your classifier will be tested against some hidden data from the same source as the original. The accuracy (percentage of classifications correct) will be calculated, then benchmarked against common methods. At the very high end of the grading scale, your accuracy will also be compared to the best submissions from other students (in your own cohort and others!). Your estimate from the cell above will also factor in, and you will be rewarded for being close to your actual accuracy (overestimates and underestimates will be treated the same).

#### Test Cell
The following code will run your classifier against the provided test data. To enable it, set the constant `SKIP_TESTS` to `False`.

The original skeleton code above classifies every row as ham, but once you have written your own classifier you can run this cell again to test it. So long as your code sets up a variable called `classifier` with a method called `predict`, the test code will be able to run. 

Of course you may wish to test your classifier in additional ways, but you *must* ensure this version still runs before submitting.

**IMPORTANT**: you must set `SKIP_TESTS` back to `True` before submitting this file!

In [57]:
SKIP_TESTS = True

if not SKIP_TESTS:
    testing_spam = np.loadtxt(open("data/testing_spam.csv"), delimiter=",").astype(np.int)
    test_data = testing_spam[:, 1:]
    test_labels = testing_spam[:, 0]
    
    predictions = classifier.predict(test_data)
    print(predictions)
    accuracy = np.count_nonzero(predictions == test_labels)/test_labels.shape[0]
    print(f"Accuracy on test data is: {accuracy}")
    

In [62]:
import sys
import pathlib

fail = False;

if not SKIP_TESTS:
    fail = True;
    print("You must set the SKIP_TESTS constant to True in the cell above.")
    
p3 = pathlib.Path('./spamclassifier.ipynb')
if not p3.is_file():
    fail = True
    print("This notebook file must be named spamclassifier.ipynb")
    
if "create_classifier" not in dir():
    fail = True;
    print("You must include a function called create_classifier.")

if "my_accuracy_estimate" not in dir():
    fail = True;
    print("You must include a function called my_accuracy_estimate.")
else:
    if my_accuracy_estimate() == 0.5:
        print("Warning:")
        print("You do not seem to have provided an accuracy estimate, it is set to 0.5.")
        print("This is the actually the worst possible accuracy – if your classifier")
        print("got 0.1 then it could invert its results to get 0.9!")
    
print("INFO: Make sure you follow the instructions on the assignment page to submit your video.")
print("Failing to include this could result in an overall grade of zero for both parts.")
print()

if fail:
    sys.stderr.write("Your submission is not ready! Please read and follow the instructions above.")
else:
    print("All checks passed. When you are ready to submit, upload the notebook and readme file to the")
    print("assignment page, without changing any filenames.")
    print()
    print("If you need to submit multiple files, you can archive them in a .zip file. (No other format.)")

INFO: Make sure you follow the instructions on the assignment page to submit your video.
Failing to include this could result in an overall grade of zero for both parts.

All checks passed. When you are ready to submit, upload the notebook and readme file to the
assignment page, without changing any filenames.

If you need to submit multiple files, you can archive them in a .zip file. (No other format.)


In [59]:
# This is a test cell. Please do not modify or delete.